In [0]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
from nltk.tokenize import word_tokenize
% %tensorflow_version 1.x
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Lambda, Reshape, add, dot, Activation, LSTM, GRU, Bidirectional,\
RepeatVector, Concatenate, Dot, Subtract, TimeDistributed, Multiply
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam, RMSprop
import keras.backend as K
from pylab import *
import pandas as pd
import string
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split 

if len(K.tensorflow_backend._get_available_gpus()) > 0:
    from keras.layers import CuDNNLSTM as LSTM
    from keras.layers import CuDNNGRU as GRU

TensorFlow 1.x selected.


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.






In [0]:
MAX_QUESTION_LENGTH = 30
GLOVE_EMBED = 50
LATENT_DIM_1 = 150
LATENT_DIM_2 = 80
LATENT_DIM_3 = 30

In [0]:
prefix = '/content/drive/My Drive/Quora Questions Project/' 

In [0]:
lemmatizer = WordNetLemmatizer() 
data = pd.read_csv(prefix+"questions.csv")
data = data.dropna() # Remove null values

In [0]:
data = data.drop(['id','qid1','qid2'],axis=1)

In [0]:
data.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
g = data.groupby('is_duplicate', group_keys=False)
data = g.apply(lambda x: x.sample(g.size().min()).reset_index()).sample(frac=0.8)

In [0]:
data = column_stack((data.question1,data.question2,data.is_duplicate))
train_data, test_data = train_test_split(data, test_size= 0.15)
embeddings_index = {}
f = open(prefix+'glove.6B.50d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vectors representing the embedding for the word
    embeddings_index[word] = coefs
f.close() 

In [0]:
def preProcess(doc):
    processed = []
    for i in range(2):
        doc_processed = (doc[i]).lower()
        translator = str.maketrans('','',string.punctuation)
        doc_processed = doc_processed.translate(translator)
        doc_processed = word_tokenize(doc_processed)
        if len(doc_processed)>MAX_QUESTION_LENGTH: # Restrict question size to 80 words
            return []
        #doc_processed = [lemmatizer.lemmatize(w) for w in doc_processed]
        vect = []
        for word in doc_processed:
            if word in embeddings_index:
                vect.append(embeddings_index[word])
            else:
                vect.append(np.zeros(GLOVE_EMBED))
        vect.extend([np.zeros(GLOVE_EMBED)]*(MAX_QUESTION_LENGTH-len(vect)))
        processed.append(vect)
        #processed.extend([np.zeros(GLOVE_EMBED)]*(MAX_QUESTION_LENGTH-len(vect)))
    processed.append(doc[2])
    return processed

print(train_data[0,:])
# print(preProcess(train_data[0,:]))  

['Where can I run C++ programs online?'
 'How can I run C++ programs online?' 1]


In [0]:
train_vecs = []
for i in range(len(train_data)):
    train_vecs.append(preProcess(train_data[i,:])) 
train_vecs = [x for x in train_vecs if x] # remove all empty lists

# Preparing data for passing into keras model
X_train_1 = []
X_train_2 = []
y_train = []

for i in range(len(train_vecs)):
    X_train_1.append(train_vecs[i][0])
    X_train_2.append(train_vecs[i][1])
    y_train.append(train_vecs[i][2])

X_train_1 = np.array(X_train_1)
X_train_2 = np.array(X_train_2)
y_train = np.array(y_train)

In [0]:
test_vecs = []
for i in range(len(test_data)):
    test_vecs.append(preProcess(test_data[i,:])) 
test_vecs = [x for x in test_vecs if x] # remove all empty lists

# Preparing data for passing into keras model
X_test_1 = []
X_test_2 = []
y_test = []

for i in range(len(test_vecs)):
    X_test_1.append(test_vecs[i][0])
    X_test_2.append(test_vecs[i][1])
    y_test.append(test_vecs[i][2])

X_test_1 = np.array(X_test_1)
X_test_2 = np.array(X_test_2)
y_test = np.array(y_test)

In [0]:
# Model
LATENT_DIM_1 = 25
# Taking glove embedding inputs
input_1 = Input(shape=(MAX_QUESTION_LENGTH, GLOVE_EMBED))
input_2 = Input(shape=(MAX_QUESTION_LENGTH, GLOVE_EMBED))

# 2 Bidirectional LSTM layers for each question
lstm_11 = Bidirectional(LSTM(LATENT_DIM_1, return_sequences = True))(input_1)
print(lstm_11.shape)
# lstm_21 = Bidirectional(LSTM(LATENT_DIM_2, return_sequences = True))(lstm_11)
# print(lstm_21.shape)
# lstm_31 = Bidirectional(LSTM(LATENT_DIM_3, return_sequences = True))(lstm_21)
# print(lstm_31.shape)
avg_1 = Lambda(lambda x:K.mean(x, axis=1))(lstm_11)
avg_1 = Reshape((50,))(avg_1)
print(avg_1.shape)

lstm_12 = Bidirectional(LSTM(LATENT_DIM_1, return_sequences = True))(input_2)
# lstm_22 = Bidirectional(LSTM(LATENT_DIM_2, return_sequences = True))(lstm_12)
# lstm_32 = Bidirectional(LSTM(LATENT_DIM_3, return_sequences = True))(lstm_22)
avg_2 = Lambda(lambda x:K.mean(x, axis=1))(lstm_12)
avg_2 = Reshape((50,))(avg_2)

diff = Subtract()([avg_1, avg_2])
print(diff.shape)
dot_layer = Multiply()([avg_1,avg_2])
print(dot_layer.shape)
sub = Concatenate(axis=1)([avg_1, avg_2,diff,dot_layer])
print(sub.shape)
dense = Dense(50, activation='tanh')(sub)
print(dense.shape) 
dense = Dense(1, activation='sigmoid')(sub)
print(dense.shape) 

model = Model([input_1, input_2],dense)
model.compile(optimizer=Adam(lr=0.003), loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(input_array[:12000], target_array[:12000], batch_size = 256, epochs=20, validation_split=0.2)

(?, 30, 50)
(?, 50)
(?, 50)
(?, 50)
(?, 200)
(?, 50)
(?, 1)


In [0]:
model.fit([X_train_1,X_train_2], 1-y_train, batch_size = 256, epochs=15, validation_split=0.1) 

In [0]:
model.save_weights('quora_1.h5')

In [0]:
y_preds = model.predict([X_test_1, X_test_2])

In [0]:
crr=0
thresh=.51
for i in range(len(y_preds)):
    if y_preds[i]>=thresh:
        if 1-y_test[i]==1:
            crr+=1
    else:
        if 1-y_test[i]==0:
            crr+=1
print("Accuracy = {}%".format(100*np.round(crr/len(y_preds),4)))

Accuracy = 75.51%


In [0]:
# # Model

# # Taking glove embedding inputs
# input_1 = Input(shape=(MAX_QUESTION_LENGTH, GLOVE_EMBED))
# input_2 = Input(shape=(MAX_QUESTION_LENGTH, GLOVE_EMBED))

# # 2 Bidirectional LSTM layers for each question
# lstm_11, h,c = LSTM(20, return_state = True)(input_1)
# print(lstm_11.shape)
# # lstm_21 = Bidirectional(LSTM(LATENT_DIM_2, return_sequences = True))(lstm_11)
# # print(lstm_21.shape)
# # lstm_31 = Bidirectional(LSTM(LATENT_DIM_3, return_sequences = True))(lstm_21)
# # print(lstm_31.shape)

# lstm_12,h,c = LSTM(20, return_state = True)(input_2)
# # lstm_22 = Bidirectional(LSTM(LATENT_DIM_2, return_sequences = True))(lstm_12)
# # lstm_32 = Bidirectional(LSTM(LATENT_DIM_3, return_sequences = True))(lstm_22)

# sub = Concatenate(axis=1)([lstm_11, lstm_12])
# print(sub.shape) 
# dense = TimeDistributed(Dense(1, activation='tanh'))(sub) 
# print(dense.shape)
# dense = Reshape((60,))(dense)
# print(dense.shape)
# dense = Dense(30, activation='tanh')(dense)
# print(dense.shape) 
# dense = Dense(1, activation='tanh')(dense)
# print(dense.shape) 

# model = Model([input_1, input_2],dense)
# model.compile(optimizer=RMSprop(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])
# # model.fit(input_array[:12000], target_array[:12000], batch_size = 256, epochs=20, validation_split=0.2)
# # RMSprop(lr=0.0005)
# model.fit([X_train_1,X_train_2], 1-y_train, batch_size = 256, epochs=20, validation_split=0.2)
